**Polars**

In [1]:
import polars as pl
import pandas as pd
import bottleneck as bn
import numpy as np

Считаем датасет из файла train.csv

In [2]:
df_polars = pl.read_csv('train.csv')
print("Датасет загружен с помощью Polars:")
print(df_polars.head())
print()

Датасет загружен с помощью Polars:
shape: (5, 12)
┌─────────────┬──────────┬────────┬───────────────────┬───┬───────────┬─────────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name              ┆ … ┆ Ticket    ┆ Fare    ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---               ┆   ┆ ---       ┆ ---     ┆ ---   ┆ ---      │
│ i64         ┆ i64      ┆ i64    ┆ str               ┆   ┆ str       ┆ f64     ┆ str   ┆ str      │
╞═════════════╪══════════╪════════╪═══════════════════╪═══╪═══════════╪═════════╪═══════╪══════════╡
│ 1           ┆ 0        ┆ 3      ┆ Braund, Mr. Owen  ┆ … ┆ A/5 21171 ┆ 7.25    ┆ null  ┆ S        │
│             ┆          ┆        ┆ Harris            ┆   ┆           ┆         ┆       ┆          │
│ 2           ┆ 1        ┆ 1      ┆ Cumings, Mrs.     ┆ … ┆ PC 17599  ┆ 71.2833 ┆ C85   ┆ C        │
│             ┆          ┆        ┆ John Bradley (Fl… ┆   ┆           ┆         ┆       ┆          │
│ 3           ┆ 1        ┆ 3      ┆ Heikk

Основная информация о датасете: информация о типах данных, число пропусков, средние значения и т.д.

In [3]:
print("Основная информация о датасете:")
print("Типы данных:", df_polars.dtypes)
print("Количество пропусков:", df_polars.null_count())
print("Форма датафрейма:", df_polars.shape)
print("Описательная статистика:")
print(df_polars.describe())
print()

Основная информация о датасете:
Типы данных: [Int64, Int64, Int64, String, String, Float64, Int64, Int64, String, Float64, String, String]
Количество пропусков: shape: (1, 12)
┌─────────────┬──────────┬────────┬──────┬───┬────────┬──────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name ┆ … ┆ Ticket ┆ Fare ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---  ┆   ┆ ---    ┆ ---  ┆ ---   ┆ ---      │
│ u32         ┆ u32      ┆ u32    ┆ u32  ┆   ┆ u32    ┆ u32  ┆ u32   ┆ u32      │
╞═════════════╪══════════╪════════╪══════╪═══╪════════╪══════╪═══════╪══════════╡
│ 0           ┆ 0        ┆ 0      ┆ 0    ┆ … ┆ 0      ┆ 0    ┆ 687   ┆ 2        │
└─────────────┴──────────┴────────┴──────┴───┴────────┴──────┴───────┴──────────┘
Форма датафрейма: (891, 12)
Описательная статистика:
shape: (9, 13)
┌────────────┬─────────────┬──────────┬──────────┬───┬───────────┬───────────┬───────┬──────────┐
│ statistic  ┆ PassengerId ┆ Survived ┆ Pclass   ┆ … ┆ Ticket    ┆ Fare      ┆ Cabin

Количество пассажиров каждого класса (Pclass)

In [4]:
pclass_counts = df_polars.get_column('Pclass').value_counts().sort('Pclass')
print("Количество пассажиров по классам:")
print(pclass_counts)
print()

Количество пассажиров по классам:
shape: (3, 2)
┌────────┬───────┐
│ Pclass ┆ count │
│ ---    ┆ ---   │
│ i64    ┆ u32   │
╞════════╪═══════╡
│ 1      ┆ 216   │
│ 2      ┆ 184   │
│ 3      ┆ 491   │
└────────┴───────┘



Количество выживших мужчин и женщин на корабле

In [5]:
survived_by_sex = df_polars.group_by('Sex').agg(
    pl.col('Survived').sum().alias('survived_count')
)
print("Количество выживших по полу:")
print(survived_by_sex)
print()

Количество выживших по полу:
shape: (2, 2)
┌────────┬────────────────┐
│ Sex    ┆ survived_count │
│ ---    ┆ ---            │
│ str    ┆ i64            │
╞════════╪════════════════╡
│ male   ┆ 109            │
│ female ┆ 233            │
└────────┴────────────────┘



Пассажиры, возраст которых больше 44 лет

In [6]:
passengers_over_44 = df_polars.filter(pl.col('Age') > 44)
print(f"Пассажиры старше 44 лет ({len(passengers_over_44)} человек):")
print(passengers_over_44.select(['Name', 'Age', 'Pclass', 'Sex']))
print()


Пассажиры старше 44 лет (115 человек):
shape: (115, 4)
┌─────────────────────────────────┬──────┬────────┬────────┐
│ Name                            ┆ Age  ┆ Pclass ┆ Sex    │
│ ---                             ┆ ---  ┆ ---    ┆ ---    │
│ str                             ┆ f64  ┆ i64    ┆ str    │
╞═════════════════════════════════╪══════╪════════╪════════╡
│ McCarthy, Mr. Timothy J         ┆ 54.0 ┆ 1      ┆ male   │
│ Bonnell, Miss. Elizabeth        ┆ 58.0 ┆ 1      ┆ female │
│ Hewlett, Mrs. (Mary D Kingcome… ┆ 55.0 ┆ 2      ┆ female │
│ Wheadon, Mr. Edward H           ┆ 66.0 ┆ 2      ┆ male   │
│ Harper, Mrs. Henry Sleeper (My… ┆ 49.0 ┆ 1      ┆ female │
│ …                               ┆ …    ┆ …      ┆ …      │
│ Daly, Mr. Peter Denis           ┆ 51.0 ┆ 1      ┆ male   │
│ Swift, Mrs. Frederick Joel (Ma… ┆ 48.0 ┆ 1      ┆ female │
│ Beckwith, Mrs. Richard Leonard… ┆ 47.0 ┆ 1      ┆ female │
│ Vander Cruyssen, Mr. Victor     ┆ 47.0 ┆ 3      ┆ male   │
│ Potter, Mrs. Thomas Jr (Lily

**Ускорение работы с pandas**

Считываем датасет из файла train.csv (это данные о выживаемости на Титанике) с помощью pandas

In [7]:
df_pandas = pd.read_csv('train.csv')

Средний возраст пассажиров и его стандартное отклонение с помощью bottleneck



In [8]:
mean_age = bn.nanmean(df_pandas['Age'].values)
std_age = bn.nanstd(df_pandas['Age'].values)
print(f"Средний возраст пассажиров: {mean_age:.2f}")
print(f"Стандартное отклонение возраста: {std_age:.2f}")
print()

Средний возраст пассажиров (bottleneck): 29.70
Стандартное отклонение возраста (bottleneck): 14.52



Для каждого пассажира умножьте значение столбца Fare на 1.3 и сохраните результаты как новый столбец Fare_new

In [9]:
df_pandas['Fare_new'] = 0.0
for row in df_pandas.itertuples():
    df_pandas.at[row.Index, 'Fare_new'] = row.Fare * 1.3

print("Первые 5 строк с новым столбцом Fare_new:")
print(df_pandas[['Fare', 'Fare_new']].head())
print()

Первые 5 строк с новым столбцом Fare_new:
      Fare  Fare_new
0   7.2500   9.42500
1  71.2833  92.66829
2   7.9250  10.30250
3  53.1000  69.03000
4   8.0500  10.46500



**Оптимизация типов pandas**

Считаем датасет из файла Housing.csv (это данные о ценах домов) с помощью pandas

In [10]:
df_housing = pd.read_csv('Housing.csv')
print("Датасет Housing загружен:")
print(df_housing.info(memory_usage='deep'))
print()

Датасет Housing загружен:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 221.9 KB
None



Оптимальный с точки зрения потребления памяти тип данных - напишите свои выводы в комментариях

In [24]:
optimized_dtypes = {}

for column in df_housing.columns:
    current_dtype = df_housing[column].dtype
    memory_usage = df_housing[column].memory_usage(deep=True)
    unique_count = df_housing[column].nunique()

    print(f"\nСтолбец: {column}")
    print(f"  Текущий тип: {current_dtype}")
    print(f"  Потребление памяти: {memory_usage:,} байт")
    print(f"  Уникальных значений: {unique_count} из {len(df_housing)}")

    # Определение оптимального типа
    if df_housing[column].dtype == 'object':
        # Для категориальных данных
        unique_ratio = unique_count / len(df_housing)
        if unique_ratio < 0.5:  # Если менее 50% уникальных значений
            optimal_type = 'category'
            print(f"  Рекомендуемый тип: {optimal_type} (категориальный, {unique_ratio:.1%} уникальных)")
            optimized_dtypes[column] = 'category'
        else:
            optimal_type = 'object'
            print(f"  Рекомендуемый тип: {optimal_type} (оставить как есть, {unique_ratio:.1%} уникальных)")
            optimized_dtypes[column] = 'object'

    elif df_housing[column].dtype in ['int64', 'float64']:
        # Для числовых данных
        col_min = df_housing[column].min()
        col_max = df_housing[column].max()

        if df_housing[column].dtype == 'int64':
            if col_min >= 0:
                if col_max < 256:
                    optimal_type = 'uint8'
                elif col_max < 65536:
                    optimal_type = 'uint16'
                elif col_max < 4294967296:
                    optimal_type = 'uint32'
                else:
                    optimal_type = 'uint64'
            else:
                if col_min > -128 and col_max < 127:
                    optimal_type = 'int8'
                elif col_min > -32768 and col_max < 32767:
                    optimal_type = 'int16'
                elif col_min > -2147483648 and col_max < 2147483647:
                    optimal_type = 'int32'
                else:
                    optimal_type = 'int64'
        else:
            # Для float - используем float32 если точности достаточно
            optimal_type = 'float32'

        print(f"  Рекомендуемый тип: {optimal_type}")
        print(f"  Диапазон значений: {col_min} - {col_max}")
        optimized_dtypes[column] = optimal_type


Столбец: price
  Текущий тип: int64
  Потребление памяти: 4,492 байт
  Уникальных значений: 219 из 545
  Рекомендуемый тип: uint32
  Диапазон значений: 1750000 - 13300000

Столбец: area
  Текущий тип: int64
  Потребление памяти: 4,492 байт
  Уникальных значений: 284 из 545
  Рекомендуемый тип: uint16
  Диапазон значений: 1650 - 16200

Столбец: bedrooms
  Текущий тип: int64
  Потребление памяти: 4,492 байт
  Уникальных значений: 6 из 545
  Рекомендуемый тип: uint8
  Диапазон значений: 1 - 6

Столбец: bathrooms
  Текущий тип: int64
  Потребление памяти: 4,492 байт
  Уникальных значений: 4 из 545
  Рекомендуемый тип: uint8
  Диапазон значений: 1 - 4

Столбец: stories
  Текущий тип: int64
  Потребление памяти: 4,492 байт
  Уникальных значений: 4 из 545
  Рекомендуемый тип: uint8
  Диапазон значений: 1 - 4

Столбец: mainroad
  Текущий тип: object
  Потребление памяти: 28,395 байт
  Уникальных значений: 2 из 545
  Рекомендуемый тип: category (категориальный, 0.4% уникальных)

Столбец: guest

**ВЫВОД**

**Анализ исходного датафрейма показал:**

**Число строк:** 545  
**Число столбцов:** 13  

**Числовые признаки** (price, area, bedrooms, bathrooms, stories, parking) представлены как int64  
**Остальные признаки** — строковые (object), при этом большинство содержит 2–3 уникальных значения

На основе этого оптимальные типы данных следующие:

**price: uint32**  
Целочисленный признак. Значения цены (1.75M - 13.3M) значительно меньше предела uint32 (4.3B), поэтому использование int64 избыточно. Поскольку цены не могут быть отрицательными, оптимально использовать беззнаковый тип.

**area: uint16**  
Площадь — целые числа в диапазоне 1650-16200, что полностью укладывается в uint16 (0-65535). Это уменьшает потребление памяти на 75% по сравнению с int64.

**bedrooms, bathrooms, stories: uint8**  
Эти признаки имеют очень маленькое количество уникальных значений (от 2 до 6). Диапазон uint8 (0-255) полностью покрывает реальные значения (1-6 для bedrooms, 1-4 для bathrooms и stories), что позволяет уменьшить память с 8 байт на значение до 1 байта.

**parking: uint8**  
Количество парковочных мест имеет значения от 0 до 3, что идеально вписывается в uint8. Экономия памяти составляет 87.5%.

**mainroad, guestroom, basement, hotwaterheating, airconditioning, prefarea : category**  
Эти бинарные признаки содержат всего 2 уникальных значения при 545 строках (0.4% уникальности). Тип category хранит их как словарь категорий + числовые коды вместо строк, что даёт существенную экономию памяти по сравнению с типом object.

**furnishingstatus: category**  
Признак уровня меблировки содержит 3 уникальных значения (0.6% уникальности). Преобразование в category значительно сократит потребление памяти за счет внутреннего кодирования.

Поменяйте типы данных столбцов датафрейма на выбранные вами в прошлом пункте и сравните потребление памяти до и после оптимизации

In [26]:

# Потребление памяти до оптимизации
memory_before = df_housing.memory_usage(deep=True).sum()
memory_per_column_before = df_housing.memory_usage(deep=True)

print(f"\nПОТРЕБЛЕНИЕ ПАМЯТИ ДО ОПТИМИЗАЦИИ:")
print(f"Общее потребление памяти: {memory_before:,} байт ({memory_before/1024/1024:.2f} MB)")
print("\nПотребление по столбцам:")
for col in df_housing.columns:
    mem_usage = df_housing[col].memory_usage(deep=True)
    print(f"  {col}: {mem_usage:,} байт")

# Применение оптимизированных типов из второго задания
df_housing_optimized = df_housing.copy()

print(f"\nПРИМЕНЕНИЕ ОПТИМИЗАЦИИ ТИПОВ ДАННЫХ:")
for column, new_dtype in optimized_dtypes.items():
    old_dtype = str(df_housing_optimized[column].dtype)
    try:
        df_housing_optimized[column] = df_housing_optimized[column].astype(new_dtype)
        print(f"  ✓ {column}: {old_dtype} → {new_dtype}")
    except Exception as e:
        print(f"  ✗ {column}: ошибка преобразования {old_dtype} → {new_dtype} ({e})")

# Потребление памяти после оптимизации
memory_after = df_housing_optimized.memory_usage(deep=True).sum()
memory_per_column_after = df_housing_optimized.memory_usage(deep=True)

print(f"\nПОТРЕБЛЕНИЕ ПАМЯТИ ПОСЛЕ ОПТИМИЗАЦИИ:")
print(f"Общее потребление памяти: {memory_after:,} байт ({memory_after/1024/1024:.2f} MB)")
print("\nПотребление по столбцам после оптимизации:")
for col in df_housing_optimized.columns:
    mem_usage = df_housing_optimized[col].memory_usage(deep=True)
    print(f"  {col}: {mem_usage:,} байт")

print("ДЕТАЛЬНОЕ СРАВНЕНИЕ:")

memory_saved = memory_before - memory_after
memory_saved_percent = (memory_saved / memory_before) * 100

print(f"ОБЩЕЕ ПОТРЕБЛЕНИЕ ПАМЯТИ:")
print(f"  До оптимизации:    {memory_before:,} байт ({memory_before/1024/1024:.2f} MB)")
print(f"  После оптимизации: {memory_after:,} байт ({memory_after/1024/1024:.2f} MB)")
print(f"  Экономия:          {memory_saved:,} байт ({memory_saved/1024/1024:.2f} MB)")
print(f"  Процент экономии:  {memory_saved_percent:.1f}%")

print(f"\nСРАВНЕНИЕ ПО СТОЛБЦАМ:")
print("Столбец              | До (байт)     | После (байт)  | Экономия")
print("-" * 65)
for col in df_housing.columns:
    mem_before = memory_per_column_before[col]
    mem_after = memory_per_column_after[col]
    saved = mem_before - mem_after
    saved_percent = (saved / mem_before * 100) if mem_before > 0 else 0
    print(f"{col:20} | {mem_before:12,} | {mem_after:12,} | {saved:+,} ({saved_percent:+.1f}%)")

print(f"\nТИПЫ ДАННЫХ ДО И ПОСЛЕ ОПТИМИЗАЦИИ:")
print("Столбец              | До           | После")
print("-" * 45)
for col in df_housing.columns:
    old_dtype = str(df_housing[col].dtype)
    new_dtype = str(df_housing_optimized[col].dtype)
    print(f"{col:20} | {old_dtype:12} | {new_dtype:12}")

# Проверка, что данные не изменились
print(f"\nПРОВЕРКА ЦЕЛОСТНОСТИ ДАННЫХ:")
print(f"  Форма датафрейма не изменилась: {df_housing.shape == df_housing_optimized.shape}")
print(f"  Количество строк: {df_housing_optimized.shape[0]}")
print(f"  Количество столбцов: {df_housing_optimized.shape[1]}")


ПОТРЕБЛЕНИЕ ПАМЯТИ ДО ОПТИМИЗАЦИИ:
Общее потребление памяти: 227,244 байт (0.22 MB)

Потребление по столбцам:
  price: 4,492 байт
  area: 4,492 байт
  bedrooms: 4,492 байт
  bathrooms: 4,492 байт
  stories: 4,492 байт
  mainroad: 28,395 байт
  guestroom: 28,024 байт
  basement: 28,118 байт
  hotwaterheating: 27,952 байт
  airconditioning: 28,099 байт
  parking: 4,492 байт
  prefarea: 28,055 байт
  furnishingstatus: 33,233 байт

ПРИМЕНЕНИЕ ОПТИМИЗАЦИИ ТИПОВ ДАННЫХ:
  ✓ price: int64 → uint32
  ✓ area: int64 → uint16
  ✓ bedrooms: int64 → uint8
  ✓ bathrooms: int64 → uint8
  ✓ stories: int64 → uint8
  ✓ mainroad: object → category
  ✓ guestroom: object → category
  ✓ basement: object → category
  ✓ hotwaterheating: object → category
  ✓ airconditioning: object → category
  ✓ parking: int64 → uint8
  ✓ prefarea: object → category
  ✓ furnishingstatus: object → category

ПОТРЕБЛЕНИЕ ПАМЯТИ ПОСЛЕ ОПТИМИЗАЦИИ:
Общее потребление памяти: 10,952 байт (0.01 MB)

Потребление по столбцам после опт